# Libraries

In [1]:
import os
from crewai import Agent, Task, Crew, Process
import crewai
from dotenv import load_dotenv
from crewai import LLM
from datetime import datetime
import time
import random
import pandas as pd

In [3]:
pip show crewai

Name: crewai
Version: 0.63.2
Summary: Cutting-edge framework for orchestrating role-playing, autonomous AI agents. By fostering collaborative intelligence, CrewAI empowers agents to work together seamlessly, tackling complex tasks.
Home-page: https://crewai.com
Author: Joao Moura
Author-email: joao@crewai.com
License: 
Location: c:\Users\VICTUS\AppData\Local\Programs\Python\Python311\Lib\site-packages
Requires: appdirs, auth0-python, click, embedchain, instructor, json-repair, jsonref, langchain, litellm, openai, opentelemetry-api, opentelemetry-exporter-otlp-proto-http, opentelemetry-sdk, poetry, pydantic, python-dotenv, regex
Required-by: 
Note: you may need to restart the kernel to use updated packages.


# LLM setting

In [4]:
load_dotenv()
gpt = LLM(
    model="gpt-4o-mini-2024-07-18",
    temperature=0.0,
    top_p=0.1,
    api_key=os.getenv("OPENAI_API_KEY")
)

# Agent and Task: Financial Analyst

In [6]:
financial_analyst_agent = Agent(
    role='Financial Analyst',
    goal="""
    - Using extensive experience in the Thai financial market, analyze the news headline: {news_headline} about the Thai stock: {stock_name} and identify whether it is good, bad, or neutral news about that stock.
    """,
    verbose=True,
    backstory="""
    - Specialist with extensive experience in the Thai financial market.
    - Expert with a deep understanding of financial knowledge and analytical skills in the Thai financial market.
    - Experienced in analyzing stock news headlines to assess sentiment.
    - Expert in the Thai language, enabling accurate interpretation and analysis of news headlines.
""",
    llm=gpt, 
    memory=False
)


financial_analyst_task = Task(
    description=
    """
    - Analyze the news headline: '{news_headline}' about the Thai stock: '{stock_name}' and identify the sentiment of the news headline as either good, bad, or neutral. The sentiment should be based on whether the news would encourage investors to invest in that stock. If the news headline: '{news_headline}' is not related to the stock: '{stock_name}' at all, mark it as 'neutral'.
    """,

    expected_output=
    """
    - A JSON object in this format:
        "news_headline": {news_headline},
        "stock_name": "{stock_name}",
        "headline_sentiment": "<headline_sentiment>" in the format of only "good," "bad," or "neutral.
    """,
    
    agent=financial_analyst_agent
)

# Usage

In [7]:
def filter_by_month_and_year(month: int, year: int):

    df = pd.read_csv("news_headline.csv", parse_dates=['date'])

    filtered_df = df[(df['date'].dt.year == year) &
                     (df['date'].dt.month == month)]

    filtered_data = filtered_df[['news_headline', 'stock_symbol']]

    return filtered_data


crew = Crew(
    agents=[financial_analyst_agent],
    tasks=[financial_analyst_task],
    process=Process.sequential  
)


def run(month: int, year: int):
    filtered_data = filter_by_month_and_year(month, year)
    print(f'{filtered_data.shape}')
    for index, row in filtered_data.iterrows():
        print(
            f"Processing row: {index}, Stock Symbol: {row['stock_symbol']}, News Headline: {row['news_headline']}")
        time.sleep(random.uniform(2, 4))
        result = crew.kickoff(
            inputs={'news_headline': row['news_headline'], 'stock_name': row['stock_symbol']})
        print(result)
        now = datetime.now()
        current = now.strftime("%Y-%m-%d %H:%M:%S")
        print(current)

In [ ]:
# Example
run(10,2024)